# Handling Categorical Data
## Bars条形图
### Basic
Bokeh make it simple to create basic bar charts using the`hbar()`and `vbar()` glyphs methods.

In [1]:
fruits=['Apples','Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']

```python
p = figure(x_range=fruits, ... )
```
or like this:
```python
p = figure(x_range=FactorRange(factors=fruits), ... )
```

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import FactorRange

output_notebook()


fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
counts = [5, 3, 4, 2, 4, 6]

p = figure(x_range=FactorRange(*fruits), plot_height=250, title="Fruit Counts",
           toolbar_location=None, tools="")

p.vbar(x=fruits, top=counts, width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

Loading BokehJS ...

### Sorted排序

In [10]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
counts = [5, 3, 4, 2, 4, 6]

# sorting the bars means sorting the range factors
sorted_fruits = sorted(fruits, key=lambda x: counts[fruits.index(x)])

p = figure(x_range=sorted_fruits, plot_height=350, title="Fruit Counts",
           toolbar_location=None, tools="")

p.vbar(x=fruits, top=counts, width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

## Filled条状图填充
### Colors

第一种方式是直接将颜色与内容一一对应，就像下面创建一个字典直接在dict后面导入color，再在`vabr()`里显示颜色

In [21]:
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
counts = [5, 3, 4, 2, 4, 6]

source = ColumnDataSource(data=dict(fruits=fruits, counts=counts, color=Spectral6))

p = figure(x_range=fruits, y_range=(0,9), plot_height=250, title="Fruit Counts",
           toolbar_location=None, tools="")

p.vbar(x='fruits', top='counts', width=0.9, color='color', legend_field="fruits", source=source)

p.xgrid.grid_line_color = None
p.legend.orientation = "horizontal"
p.legend.location = "top_center"

show(p)

In [22]:
dict(fruits=fruits, counts=counts, color=Spectral6)

{'fruits': ['Apples',
  'Pears',
  'Nectarines',
  'Plums',
  'Grapes',
  'Strawberries'],
 'counts': [5, 3, 4, 2, 4, 6],
 'color': ['#3288bd', '#99d594', '#e6f598', '#fee08b', '#fc8d59', '#d53e4f']}

另一种方式是创建一个<font color=red>CategoricalColorMapper</font>，利用函数`function_camp()`可以实现：
```python
factor_cmap('fruits', palette=Spectral6, factors=fruits)
```

In [25]:
from bokeh.palettes import Spectral8# 既然有8个类别，就引入8中颜色
from bokeh.transform import factor_cmap

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries','新的','也是新的']
counts = [5, 3, 4, 2, 4, 6,3,4]

source = ColumnDataSource(data=dict(fruits=fruits, counts=counts))

p = figure(x_range=fruits, plot_height=250, toolbar_location=None, title="Fruit Counts")
p.vbar(x='fruits', top='counts', width=0.9, source=source, legend_field="fruits",
       line_color='black', # 绘制条的外形轮廓颜色
       fill_color=factor_cmap('fruits', palette=Spectral8, factors=fruits))

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = 9
p.legend.orientation = "horizontal"
p.legend.location = "top_center"

show(p)

### Stack堆放

In [27]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]
colors = ["#c9d9d3", "#718dbf", "#e84d60"]

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 4, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

p = figure(x_range=fruits, plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar_stack(years,# vbar_stack函数实现堆放
             x='fruits', width=0.9, color=colors, source=data,
             legend_label=years)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

## Grouped分组
### Nested Categories嵌套类

In [29]:
from bokeh.models import ColumnDataSource, FactorRange

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar(x='x', top='counts', width=0.9, source=source)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

## Heat Maps热力图